In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Quantification of animal interactions

today, we will perform two fundamental analyses to determine if and how two animals show any attraction to one another.
### 1. determining attraction
- how can we formally show that they are attracted to one another? (spoiler: they are)
### 2. computing a neighborhood map
- let's generate a neighborhood map to get a first sense at the 'collective behavior rules' of the animals.

We will use trajectories of two fish that were recorded for two hours while swimming in an empty dish.



# Let's load trajectories of two interacting fish

The data has the following format:

x1, y1, p1, x2, y2, p2

(x,y are the coordinates of the animals 1 and 2 at each time point, p is irrelevant).

In [ ]:
tra=np.loadtxt('trajectories_day5.txt')
tra.shape

In [ ]:
#for easier access to the data of each animal, let's put the trajectories into separate variables:
an1=tra[:,[0,1]]
an2=tra[:,[3,4]]

print(an1)

In [ ]:
# As always, let's first take a look where the animals went
# for example by using an x,y, scatter plot
# you can plot the two animals on top of each other in different colors for example.
# remember that s and alpha are useful parameters for scatter when you have many points.
# you can specify color like this: color='r', for red, 'g', 'b'. google has more color codes.

plt.scatter() #animal 1
plt.scatter() #animal 2

In [ ]:
#Or plot them separately 

fig,axes =plt.subplots(1,2, sharex=True, sharey=True) #create two subplots side by side

axes[0].scatter() #plot the first animal in green
axes[0].set_aspect('equal') #set axes to equal scale
axes[1].scatter() #plot the second animal in red
axes[1].set_aspect('equal')

In [ ]:
# %load ./snippets/solution01.py
#uncomment above to load a solution


# Quantifying attraction

- It looks like both animals explored the arena more or less evenly and equally.
- Let's analyze the interaction by checking if they were closer together than chance levels

In [ ]:
# lets calculate the **observed** distance between the two animals:
# remember, this is equivalent to how we looked at speed in a previous session.
# np.sqrt()
# x**2 means x squared.

diff_12=
dist=
print(dist)

# the correct result is:
#[298.83157213 298.54594035 295.98163524 ... 199.7900971  199.03755073
# 199.52689668]

In [ ]:
# %load ./snippets/solution02.py
#uncomment above to load a solution


In [ ]:
# let's plot to take a look at the first 10000 inter animal distances
plt.plot(dist[0:10000])
plt.axhline(dist.mean(),color='g') #adding some visual guides
plt.axhline(dist.max(),color='r')

In [ ]:
#let's look at the mean distance over the entire recording:
#this corresponds to the green line above.
print(dist.mean())

In [ ]:
# how do we know that this is lower or greater than chance level?
# one option is by computing a chance level *expected* distance for comparison.
# try to understand the code below and decide what might be good values for 'nShift' and 'minShift'.


nShift=10
minShift=5000
distShift=np.zeros(nShift)
for i in range(nShift):
    shift=(i+1)*minShift
    an2_shift=np.roll(an2,shift,axis=0)
    diff_12s=an1-an2_shift
    dist_12s=np.sqrt(diff_12s[:,0]**2+diff_12s[:,1]**2)
    distShift[i]=np.mean(dist_12s)
    
print(distShift)

## How different is the expected distance from the observed distance?

In [ ]:
#Let's look at that visually using distributions:
plt.hist(dist,bins=100); #histogram of observed distances across all frames.
plt.hist(dist_12s,bins=100,alpha=0.5); #histogram of shuffled distances.

In [ ]:
#Let's compare at all the shifted distances to the observed:
#for this we will use another data structure:
# the pandas dataframe:

allDist=np.append(distShift,dist.mean())
df=pd.DataFrame(allDist,columns=['dist'])
df['type']='shifted'
df.loc[10,'type']='observed'
df


In [ ]:
# Once the data is in this table format, seaborn can conveniently generate summary plots:
# The logic is then similar to R 'tapply'
sns.stripplot(data=df,x='type',y='dist')
plt.ylim([0,350])

In [ ]:
#sns.pointplot is another useful plot that can generate error bars. (which are tiny here).
#you can also combine these plots. one in black, and the individual datapoints in blue, for example.

sns.pointplot(data=df,x='type',y='dist', join=False, errorbar='sd', color='k')
sns.stripplot(data=df,x='type',y='dist')
plt.ylim([0,350])

In [ ]:
#Attraction index
#Let's quantify HOW much attraction we see:

#(expected-observed)/expected

attraction=(allDist.mean()-dist.mean())/allDist.mean()
print(attraction)

In [ ]:
# What does this number mean. can you phrase this in a few words?

# Let's compute a neighborhood map!

- This means to compute egocentric coordinates for animal 1
- ... For this, we need to put animal 1 at the center of the world
- ... and rotate the world so that animal 1 faces up
- ... ... For this we need to determine the heading of animal 1
- ... ... ... let's assume the animals move only forward

# toDo:
1. compute heading of animal 1
2. compute position of animal 2 relative to animal 1 -> relativePos
3. rotate relative positions such that animal 1 always heads up -> relativePosRot
4. plot rotated position to reveal neighborhood map

# 1. compute heading of animal 1
- infer heading from direction of movement

In [ ]:
# let's start by calculating the direction animal 1 moves from frame to frame.
# this is the same approach we used last time when we talked about speed.
# remember, there is usually a way to do things on the entire trajectory at once.
# e.g. the numpy function np.diff() computes the differences between consecutive elements of a vector.

# begin by computing the differences between frames in x an y
dx = 
dy = 

#those differences can be plugged into our formula for the distance:
#note how we can again compute the entire vector at once!

distances = np.sqrt(dx**2 + dy**2) 

# (we don't really need the distances, we only care about dx and dy, but let's take a brief look)


In [ ]:
# %load ./snippets/solution1.py
#uncomment above to load a solution


In [ ]:
plt.plot(distances[:300]); #plot the distances travelled in each of the first 300 frames

OK, great. The fish swims in clear bouts.

In [ ]:
# now you can use the differences in x and y to compute IN WHICH DIRECTION they point
# this is trigonometry. Use the function np.arctan2(y, x) with the correct variables to compute the direction for each frame
# again, this is a vector operation.
# the correct result is ([ 0.43240778,  0.8834045 ,  2.28757867, ...,  0.        , -1.10714872, -0.16514868])

headings = 
print(headings)

In [ ]:
# %load ./snippets/solution2.py
#uncomment above to load a solution


In [ ]:
# let's take a look at the first 300 frames. What do we notice?
plt.plot(headings[:300])

In [ ]:
#here is a function to smooth a time series, using a moving average.
# let's not worry about the details but note you can tune the degree of smoothing using the 'window_size' parameter

def moving_average_2d(data, window_size):
    """Compute the moving average of a 2D array (data) across each column."""
    # Initialize an empty array to store the smoothed data
    smoothed_data = np.empty((data.shape[0] - window_size + 1, data.shape[1]))
    
    # Apply moving average for each column
    for i in range(data.shape[1]):  # Iterate over columns
        smoothed_data[:, i] = np.convolve(data[:, i], np.ones(window_size) / window_size, mode='valid')
    
    return smoothed_data


In [ ]:
# try a few different parameters for 'window_size'. what do you notice?
# what is your intuition for a good parameter value?

window_size=10

an1s=moving_average_2d(an1,window_size)
an2s=moving_average_2d(an2,window_size)

# Calculate differences in smoothed x and y
dxs = np.diff(an1s[:,0])
dys = np.diff(an1s[:,1])
distancess = np.sqrt(dxs**2 + dys**2)

plt.plot(distancess[:300])

In [ ]:
# bonus task: generate a plot that superimposes the result for several values of the parameter.
# hint, use a loop over multiple values. If you plot within a loop, the lines will be plotted in the same graph.

#your answer:

for i in range(5):


In [ ]:
# %load ./snippets/solution3.py
#uncomment above to load a solution


In [ ]:
#now we can re-compute the heading using the smoothed trajectory. What do you notice?
#try plotting the original head and smoothed heading on top of each other.

headingss = np.arctan2(dys, dxs)

plt.plot(headings[:300],'r')
plt.plot(headingss[:300])

# 2. compute position of animal 2, relative to animal 1

- this is very simple, any ideas?

In [ ]:

relativePos=

In [ ]:
# %load ./snippets/solution4.py
#uncomment above to load a solution


# 3. rotate relative positions such that animal 1 always heads up -> relativePosRot

- now it gets a bit more interesting:
1. transform the relative positions to polar coordinates
2. then, remember how can you rotate the fish up?

- below are two helper functions to transform coordinates from cartesian to polar and back.

In [ ]:
def cart2pol(x, y): #transform from cartesian to polar
    theta = np.arctan2(y, x)
    rho = np.hypot(x, y)
    return theta, rho

In [ ]:
def pol2cart(theta, rho): #transform from polar to cartesian
    x = rho * np.cos(theta)
    y = rho * np.sin(theta)
    return x, y

In [ ]:

#first, transform relative positions from cartesian to polar
relativePos_pol=
print('Polar coordinates: ',relativePos_pol)

#now do the rotation.
#hint: you have to combine the polar coordinates with the headings.
#but there is a catch: the vectors don't have the same length. remember why?
#just drop the last position. to use all of a vector except the last entry, use this index: v[:-1]

relativePos_pol_rot=
print('Polar coordinates rotated: ', relativePos_pol_rot)

#the result should be this:

#Polar coordinates:  (array([0.33415415, 0.32949056, 0.32485253, ..., 2.53836832, 2.53872638,
#       2.53916742]), array([302.1981665 , 303.03401754, 303.89539654, ..., 198.86808196,
#       198.91860357, 199.07792124]))
#Polar coordinates rotated:  [-2.08434974 -2.09999897 -2.26379046 ...  2.64314516  2.62977952
#  2.63689466]

In [ ]:
# %load ./snippets/solution5.py
#uncomment above to load a solution


In [ ]:
#finally, we bring the coordinates back to cartesian for plotting
#again, pay attention to the different lengths of the relevant vectors. we drop the last item where necessary

relativePos_pol_rot_cart=pol2cart(relativePos_pol_rot,relativePos_pol[1][:-1])
print(relativePos_pol_rot_cart)

oof, after some mental acrobatics, we are done with the heavy-lifting! Finally on to the fun part!

# 4. plot rotated position to reveal neighborhood map

- new that you have animal 2 in the egocentric coodinates of animal 1, how can you plot them?
- thinking back to our earlier sessions, consider these options:
1. plt.scatter hint: play with size (s) and alpha parameters for a pretty map!
2. np.histogram2d which will generate a heatmap image (google for examples)

In [ ]:
#find good parameters to scatter to reveal the neighborhood map!
plt.scatter(,s=,alpha=)
plt.gca().set_aspect('equal')
plt.axhline(0,ls=':',c='gray') #some pretty visual guides
plt.axvline(0,ls=':',c='gray')


In [ ]:
heatmap, xedges, yedges = np.histogram2d() #google the docs to understand the parameters
plt.imshow(heatmap) #plot the heatmap

In [ ]:
# %load ./snippets/solution6.py
#uncomment above to load a solution


Voila, congratulations! you can clearly see where animal 1 prefers to position itself relative to animal 2.

Do you notice a difference in orientation between the heatmap and the scatterplot?

You can read about the definition of polar coordinates and the orientation of the data in python image plots vs. data plots online. (notice the x,y axis labels on the image plots?

Below is a dummy trajectory for movements in the four cardinal directions. You can get a sense how they are transformed into heading in radians and degrees.

In [ ]:
testTra=np.array([[0,0],[0,1],[1,1],[1,0],[0,0]])
testTra

In [ ]:
testTra_d=np.squeeze(np.diff(testTra,axis=0))
testTra_d

In [ ]:
testTra_h = np.arctan2(testTra_d[:,1], testTra_d[:,0])
testTra_h

In [ ]:
np.rad2deg(testTra_h)

In [ ]:
# %load ./snippets/solution1.py
np.rad2deg(testTra_h)